<font color='red'><h1>Obtaining data from PostgreSQL and loading them as Dataframes</h1></font>

In [1]:
import pandas as pds
from sqlalchemy import create_engine
import psycopg2
import pandas.io.sql 
from tqdm import tqdm

#list of global variables
client=None
dbName=None
collectionName=None
 


In [2]:
def createDataFramefromTbl(tableName,engine):
    df=None
    connection=None
    try:
        # Create an engine instance
       
        # Connect to PostgreSQL server
        connection = engine.connect()
        df= pds.read_sql("select * from \""+tableName+"\" ", connection);
    except Exception as e:
        print('Exception in createDataFramefromTbl----->\n',e)
    finally:
        if connection is not None:
            connection.close()
    return df
    


   
   
    



<font color='red'><h1>Performing Transformation on Dataframe 2 to perform aggregation</h1></font>

In [3]:
#grouping the data in dataframe 2 using groupby and performing aggregation through Mean
def dataframe2Agg(df2):
    dataFrame2_gb_agg=df2.groupby(['name','cityId']).agg({"Air_Quality_Index":"mean",
             "Carbon_Monoxide":"mean",
            "Nitrogen_Monoxide":"mean",
            "Nitrogen_dioxide":"mean",
            "Ozone":"mean",
            "Sulphur_dioxide":"mean",
            "Fine_Particles_matter":"mean",
            "Coarse_particulate_Matter":"mean",
            "Ammonia":"mean"})
    return dataFrame2_gb_agg


<font color='red'><h1>Performing Transformation on Dataframe 3 to perform aggregation</h1></font>

In [18]:
#grouping the data in dataframe 2 using groupby and performing aggregation through Mean
def dataframe3Agg(df3):
    dataFrame3_gb_agg=df3.groupby(['city_id']).agg({"clouds":"mean",
             "dew_point":"mean",
            "feels_like":"mean",
            "humidity":"mean",
            "pressure":"mean",
            "sunrise":"mean",
            "sunset":"mean",
            "temperature":"mean",
            "uvi":"mean",
            "wind_deg":"mean",
            "wind_speed":"mean",
            "weather_title": lambda x: pd.Series.mode(x).iat[0],
            "pop": "mean"
           })
    return dataFrame3_gb_agg



<font color='red'><h1>Joining aggregated Dataframe2 and aggregated Dataframe3 through City ID and Date</h1></font>

In [9]:
def join2_3(dataFrame2_gb_agg,dataFrame3_gb_agg):
    dataFrame2_3_agg = pds.merge(dataFrame2_gb_agg, dataFrame3_gb_agg,  how='left', left_on = ['city_id'] ,right_on=['cityId'])

    #df_dummy=dataFrame4.head()
    #dataFrame4.to_csv("D:\\MSC_DAD\\Sem1\\Database and Analytics Program\\Project\\Joined_Data.csv")
    #print('Join operation has been performed')
    return dataFrame2_3_agg

<font color='red'><h1>Joining dataframe 1 and Aggregated Dataset of Dataframes 2 and 3  based on City ID </h1></font>

In [5]:
def joinFinal(dataFrame1,dataFrame2_3_agg):    
    dataFrame_final = pds.merge(dataFrame1, dataFrame2_3_agg,  how='left', left_on = ['city_id'] ,right_on=['cityId'])
    #dataFrame5.to_csv("D:\\MSC_DAD\\Sem1\\Database and Analytics Program\\Project\\Joined_Data_1.csv")
    print('Join operation has been performed')
    return dataFrame_final

<font color='red'><h1>Joining dataframes 2 and 3 based on City ID and Dates</h1></font>

In [6]:
def normalJoin_2_3(dataFrame2,dataFrame3):
    dataFrame4 = pds.merge(dataFrame2, dataFrame3,  how='left', left_on = ['date','cityId'] ,right_on=['date','city_id'])
    #df_dummy=dataFrame4.head()
    #dataFrame4.to_csv("D:\\MSC_DAD\\Sem1\\Database and Analytics Program\\Project\\Joined_Data.csv")
    print('normalJoin_2_3->Join  operation has been performed')
    return dataFrame4

<font color='red'><h1>Joining dataframes 1 and 4 based on City ID </h1></font>

In [7]:
def normalJoin_1_4(dataFrame1,dataFrame4):
    dataFrame5 = pds.merge(dataFrame1, dataFrame4,  how='left', left_on = ['city_id'] ,right_on=['cityId'])
    #dataFrame5.to_csv("D:\\MSC_DAD\\Sem1\\Database and Analytics Program\\Project\\Joined_Data_1.csv")
    print('normalJoin_1_4->Join operation has been performed')
    return dataFrame5

<font color='red'><h1>Transforming Dataframe 5 [Dataset created by joining dataset1,2 ,3 on City ID and Date] to achieve single row entry for each city  </h1></font>

In [19]:
def singleRowTranform(df):
    cityIds = df['city_id'].unique()
    count = 0
    list=[]
    keys = ['clouds',  'weather_desc', 'dew_point', 'feels_like', 'humidity', 'weather_code', 'weather_title', 'pressure', 'sunrise', 'sunset', 'temperature', 'uvi', 'wind_deg', 'wind_speed']
    resultant_df = pd.DataFrame()
    for cityId in cityIds:
        #if count == 5:
         #   break
        count += 1
        columns = df.loc[df['city_id'] == cityId]
        print('columns', len(columns))
        consolidatedObject = {}
        for index, j in columns.iteritems():
            print('j', j)
            for key in keys:
                consolidatedObject['day' + str(index + 1) + '.' + key ] = j[key]
                consolidatedObject['city_id'] = j['city_id']
        list.append(consolidatedObject)
        print('Row added to dataframe in singleRowTranform')
    resultant_df = pd.DataFrame(list)
    return resultant_df
#singleRowTranform(dataFrame4)

<font color='red'><h1>Joining Datframe constructed using aggregation with Datframe constructed by simple join and transformation to achieve single entry per city  </h1></font>

In [9]:
def mainJoin(resultant_df,dataFrame_final):
    exportdf = pds.merge(resultant_df, dataFrame_final,  how='left', left_on = ['city_id'] ,right_on=['cityId'])
    print('mainJoin->Join operation has been performed')
    return exportdf

<font color='red'><h1>Exporting the Resultant Dataset to Postgres</h1></font>

In [11]:
def exporttoPostgres(df,engine):
    try:

        
        print('Engine object is-->',engine)
        connection = engine.connect()
        df.to_sql(resultantTableName, engine,if_exists='replace',method='multi')
        


    except Exception as e:
        print('Exception when connecting to Amazon RDS->',e)


    print('Table created sucessfully in PostgreSQL')

<font color='red'><h1>Main function  for the file</h1></font>

In [16]:


def x20172176_Joining_Datasets_main(engine):

    global dataFrame2_gb_agg, dataFrame3_gb_agg

    try:
        tableName1= cityTableName
        dataFrame1=createDataFramefromTbl(tableName1,engine)

        tableName2=airQualityTableName
        dataFrame2=createDataFramefromTbl(tableName2,engine)

        tableName3=weatherTableName
        dataFrame3=createDataFramefromTbl(tableName3,engine)
        print('All 3 DataFrames loaded sucessfully------------------>')  

        #grouping the data in dataframe 2 using groupby and performing aggregation through Mean
        dataFrame2_gb_agg=dataframe2Agg(dataFrame2)
        dataFrame2_gb_agg

        print('Aggregated dataframe2 created------------------>')

        #grouping the data in dataframe 3 using groupby and performing aggregation through Mean
        dataFrame3_gb_agg=dataframe3Agg(dataFrame3)
        dataFrame3_gb_agg

        print('Aggregated dataframe3 created------------------>')
        #changing the index for the newly created dataframes
        dataFrame2_gb_agg = dataFrame2_gb_agg.reset_index()
        dataFrame3_gb_agg = dataFrame3_gb_agg.reset_index()

        dataFrame2_3_agg = join2_3(dataFrame3_gb_agg, dataFrame2_gb_agg)
        dataFrame_final=joinFinal(dataFrame1,dataFrame2_3_agg)
        dataFrame_final

        print('Final Datframe created------------------>')

        dataFrame_final.drop('city_id_y', inplace=True, axis=1)
        dataFrame_final.drop('city_id_x', inplace=True, axis=1)
        dataFrame_final.drop('object_id', inplace=True, axis=1)

        exporttoPostgres(dataFrame_final,engine)
        print('Join Operation completed------------------>')
    except Exception as e:
        print('Error while performing joins')
    
    
    